In [1]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split

caltech_dir = "./"
categories = ["corgi", "golden" ,"dach" ,"husky" ,"beagle"] # 각 카테고리 폴더에서 불러오기
nb_classes = len(categories)

image_w = 100
image_h = 100

pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

        if i % 700 == 0:
            print(cat, " : ", f)

X = np.array(X)
y = np.array(y)
#1 0 0 0 이면 airplanes
#0 1 0 0 이면 buddha 이런식


X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save("./multi_image_data.npy", xy)

print("ok", len(y))

corgi  파일 길이 :  970
corgi  :  .//corgi\corgi 10.jpg
corgi  :  .//corgi\corgi2909.jpg
golden  파일 길이 :  799
golden  :  .//golden\gold (1).jpg
golden  :  .//golden\gold_dog665.jpg
dach  파일 길이 :  945
dach  :  .//dach\dachs.jpg
dach  :  .//dach\docss (67).jpg
husky  파일 길이 :  502
husky  :  .//husky\husky1.jpg
beagle  파일 길이 :  803
beagle  :  .//beagle\beagle (1).jpg
beagle  :  .//beagle\beagle645.jpg
ok 4019


In [6]:
import os, glob, numpy as np
import tensorflow.keras as keras
# module 'tensorflow' has no attribute 'get_default_graph'
from keras.models import Model, Input
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K
from tensorflow.python.framework import ops
ops.reset_default_graph()
import tensorflow as tf
'''
config = tf.ConfigProto()  텐서플로우 2. 에서 구동안됨
module 'tensorflow' has no attribute 'ConfigProto'
'''
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

# session = tf.Session(config=config) 자동세션기능
X_train, X_test, y_train, y_test = np.load('./multi_image_data.npy',allow_pickle=True)
print(X_train.shape)
print(X_train.shape[0])

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


(3014, 100, 100, 3)
3014
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11741685945767618669
]


In [7]:
categories = ["corgi", "golden" ,"dach" ,"husky" ,"beagle"]
nb_classes = len(categories)

#일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255

print('X_train shape: {} | y_train shape: {}\nX_test shape : {} | y_test shape : {}'.format(
    X_train.shape, y_train.shape, X_test.shape, y_test.shape))


#세개의 모델에서 같은 형태의 데이터를 사용할 것, 모든 모델에서 사용할 단일 입력 레이어를 정의.
input_shape =X_train[0,:,:,:].shape
model_input = Input(shape =input_shape)

X_train shape: (3014, 100, 100, 3) | y_train shape: (3014, 5)
X_test shape : (1005, 100, 100, 3) | y_test shape : (1005, 5)


In [18]:
# 영택이 모델 + 배치정규화 버전 + 드롭아웃 삭제
from keras.layers import BatchNormalization, Dropout
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average
def young_cnn(model_input):
    
    #mlpconv block 1
    x = Conv2D(32, (3, 3), activation='relu',padding='same')(model_input)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    
   
    
        #mlpconv block2
    x = Conv2D(64, (3, 3), activation='relu',padding='same')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    
    
    
    
    #mlpconv block3
    x = Conv2D(128, (3, 3), activation='relu',padding='same')(x)
    x = Conv2D(256, (1, 1), activation='relu')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(5, (1, 1))(x)
    
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='young_cnn')
    
    return model
    
young_cnn_model =  young_cnn(model_input)
young_cnn_model.summary()

Model: "young_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 100, 100, 32)      896       
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 100, 100, 32)      1056      
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 100, 100, 32)      1056      
_________________________________________________________________
batch_normalization_10 (Batc (None, 100, 100, 32)      128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 50, 50, 64)        18

In [19]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
def compile_and_train(model, num_epochs):
    model.compile(loss = categorical_crossentropy, optimizer=Adam(), metrics=['acc'])
    history= model.fit(x=X_train, y=y_train, batch_size=32, epochs=num_epochs, verbose=1,
                       validation_split =0.2)
    
_ = compile_and_train(young_cnn_model, num_epochs=20)

Train on 2411 samples, validate on 603 samples
Epoch 1/20
2411/2411 [==============================] - 45s 19ms/step - loss: 1.3985 - acc: 0.3874 - val_loss: 1.8120 - val_acc: 0.1990
Epoch 2/20
2411/2411 [==============================] - 44s 18ms/step - loss: 1.1240 - acc: 0.5313 - val_loss: 2.1891 - val_acc: 0.1940
Epoch 3/20
2411/2411 [==============================] - 43s 18ms/step - loss: 0.9738 - acc: 0.5761 - val_loss: 2.7490 - val_acc: 0.1940
Epoch 4/20
2411/2411 [==============================] - 43s 18ms/step - loss: 0.9015 - acc: 0.6188 - val_loss: 3.4105 - val_acc: 0.1940
Epoch 5/20
2411/2411 [==============================] - 43s 18ms/step - loss: 0.7904 - acc: 0.6661 - val_loss: 2.5341 - val_acc: 0.1940
Epoch 6/20
2411/2411 [==============================] - 44s 18ms/step - loss: 0.7107 - acc: 0.7047 - val_loss: 2.2719 - val_acc: 0.1990
Epoch 7/20
2411/2411 [==============================] - 43s 18ms/step - loss: 0.7147 - acc: 0.7238 - val_loss: 1.2542 - val_acc: 0.4478
E

In [20]:
print("정확도 : %.4f" % (young_cnn_model.evaluate(X_test, y_test)[1]))

1005/1005 [==============================] - 7s 7ms/step
정확도 : 0.7612
